In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skg import nsphere_fit
import matplotlib.collections as mcoll
import matplotlib.path as mpath

%matplotlib qt

In [202]:
video_file = '../20hr-wingless-orco/20hr-wingless-orco_phase_1.mp4'
# load the first frame
cap = cv2.VideoCapture(video_file)
ret, frame = cap.read()
cap.release()


# get user input using ginput of 5 points on the image
plt.imshow(frame)
plt.title('Select 5 points on the edge of the arena')
pts = plt.ginput(5)
plt.close()

# fit a sphere to the points
true_radius = 75 # mm
FPS = 10

pts = np.array(pts)
radius, center = nsphere_fit(pts)
print('Center:', center)
print('Radius:', radius)
sf = radius / true_radius
print('Scale factor:', sf)

Center: [412.66453456 416.34544454]
Radius: 414.5000809426005
Scale factor: 5.5266677459013405


In [203]:
# get user input using ginput of 5 points on the image for trail
plt.imshow(frame)
plt.title('Select 5 points on the trail')
pts = plt.ginput(5)
plt.close()

# fit a sphere to the points
trail_radius, trail_center = nsphere_fit(pts)
# get the distance range for the trail by uniformly sampling the sphere
xs = trail_center[0] + trail_radius * np.cos(np.linspace(0, 2*np.pi, 100))
ys = trail_center[1] + trail_radius * np.sin(np.linspace(0, 2*np.pi, 100))
# get distance of each point from the arena center
distances = np.linalg.norm(np.array([xs, ys]).T - center, axis=1)/sf
# plot the distance range
plt.hist(distances, bins=20)
plt.xlabel('Distance from center (mm)')
plt.ylabel('Frequency')
plt.title('Distance range of the trail')
plt.show()


In [204]:
# get tracklets
def get_tracklets(data, center, radius):
    n_flies = data.shape[1]//6
    tracklets = []
    for i in range(n_flies):
        # get track
        track = data.iloc[:, i*6:i*6+6].copy().reset_index()
        track.columns = ['Frame','ID','x','y','body_crossinglength','body_width','heading']

        # remove nan values
        track.replace(-1, np.nan, inplace=True)
        track.dropna(inplace=True)

        # skip if the track is too short (less than 10 frames)
        if track.shape[0] < 10:
            continue

        # scale the coordinates
        track['x'] = track['x']
        track['y'] = track['y']

        # Calculate other variables (scaled to mm)
        x_ = np.concatenate(([track['x'].iloc[0]], track['x'].values))
        y_ = np.concatenate(([track['y'].iloc[0]], track['y'].values))

        # kinematics
        track['velocity'] = np.sqrt(np.diff(x_)**2 + np.diff(y_)**2) * FPS 
        track['motion_direction'] = np.arctan2(np.diff(y_), np.diff(x_))
        track['acceleration'] = np.diff(np.concatenate(([0], track['velocity'].values))) * FPS
        track['angular_velocity'] = np.diff(np.concatenate(([track['heading'].iloc[0]], track['heading'].values))) * FPS
        track['angular_acceleration'] = np.diff(np.concatenate(([0], track['angular_velocity'].values))) * FPS
        track['radial_distance'] = np.sqrt((track['x'] - center[0])**2 + (track['y'] - center[1])**2)

        # odor trail related variables
        track['distance'] = np.sqrt((track['x'] - center[0])**2 + (track['y'] - center[1])**2) - radius
        track['angle'] = np.rad2deg(np.unwrap(np.arctan2(track['y'] - center[1], track['x'] - center[0])))
        track['angular_distance'] = np.deg2rad(track['angle']) * radius
        track['trail_heading'] = track['heading'] - np.arctan2(track['y'] - center[1], track['x'] - center[0])

        # scale the variables
        track['x'] = track['x'] / sf
        track['y'] = track['y'] / sf
        track['velocity'] = track['velocity'] / sf
        track['acceleration'] = track['acceleration'] / sf
        track['radial_distance'] = track['radial_distance'] / sf
        track['distance'] = track['distance'] / sf
        track['angular_distance'] = track['angular_distance'] / sf

        tracklets.append(track)
    return tracklets


In [234]:
csv_file = '../20hr-wingless-orco/20hr-wingless-orco-phase_1.csv'
data = pd.read_csv(csv_file, header=None)
tracklets = get_tracklets(data, center, radius)

In [223]:
fig, ax = plt.subplots()
# plot the inverted image
ax.imshow(1-frame,cmap='gray_r')
# plot the tracklets
for track in tracklets:
    ax.plot(track['x']*sf, track['y']*sf, '-',alpha=0.5, color='black',linewidth=0.1)
# plot the odor trail
circle = plt.Circle(center, radius, color='r', fill=False, linewidth=2)
ax.add_artist(circle)
plt.show()
# calculate a 2d histogram of the x and y coordinates
n_bins = 100
# combine all x and y coordinates
x = np.concatenate([track['x'].values for track in tracklets])
y = np.concatenate([track['y'].values for track in tracklets])
H, xedges, yedges = np.histogram2d(x, y, bins=n_bins)
H = H.T
# log scale
H = np.log(H+1)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
fig, ax = plt.subplots()
im = ax.imshow(H, extent=extent, origin='lower', cmap='hot')
cbar = plt.colorbar(im, ax=ax)
cbar.set_ticks(np.log([1, 10, 100, 1000]))
cbar.set_ticklabels([1, 10, 100, 1000])
# plot the odor trail
circle = plt.Circle(center, radius, color='k', fill=False, linewidth=2,zorder=10)
ax.add_artist(circle)
# plot the tracklets
for track in tracklets:
    ax.plot(track['x'], track['y'], '-',alpha=0.5, color='black',linewidth=0.1)
plt.show()

In [224]:
tracklet = tracklets[0]
plt.plot(tracklet['Frame'], tracklet['velocity'])

In [235]:
# get all velocities filtering only after the first 15 minutes
max_velocity = 10*3 # mm/s (15 body lengths per second)
velocities = np.concatenate([track[track['velocity'] < max_velocity]['velocity'].values for track in tracklets])

# fit to a double exponential distribution + gaussian
from scipy.optimize import curve_fit
def exp(x, a, b):
    # half cauchy
    return a * np.exp(-b*x)
def gauss(x, a, b, c):
    return a * np.exp(-b*(x-c)**2)
def egg_model(x, a, b, c, d, e, f, g, h):
    return exp(x, a, b) + gauss(x, c, d, e) + gauss(x, f, g, h)
def eg_model(x, a, b, c, d, e):
    return exp(x, a, b) + gauss(x, c, d, e)


In [236]:
plt.figure()
# fit the data by creating a histogram and fitting the histogram
hist, bins = np.histogram(velocities, bins=100)
bins = bins[:-1] + (bins[1] - bins[0])/2
hist = hist / np.sum(hist) # normalize
# plot the histogram
plt.bar(bins, hist, width=bins[1]-bins[0], alpha=0.5, color='gray')
p0 = [0.33, 0.1] + [0.33, 0.1, 20] + [0.33, 0.1, 5]
bounds = [[0,1],[0,np.inf]] + [[0,1],[0,np.inf],[0,max_velocity]]*2
bounds = list(zip(*bounds))
popt1, pcov = curve_fit(egg_model, bins, hist, maxfev=50000, p0=p0, bounds=bounds)
pred1 = egg_model(bins, *popt1)
plt.plot(bins, pred1, 'k-', linewidth=2)
# # plot the individual components
plt.plot(bins, exp(bins, *popt1[:2]), 'k--', label='Exponential')
plt.plot(bins, gauss(bins, *popt1[2:5]), 'g--', label='Gaussian 1')
plt.plot(bins, gauss(bins, *popt1[5:]), 'b--', label='Gaussian 2')
plt.xlabel('Velocity (mm/s)')
plt.ylabel('Probability')
plt.yscale('log')
plt.title('Velocity distribution (Exp + 2 Gaussians)')
plt.ylim([1e-3, 1])
plt.legend(frameon=False)
plt.savefig('figures/velocity_distribution_exp_2_gaussians.png')
plt.show()

plt.figure()
# fit the data by creating a histogram and fitting the histogram
hist, bins = np.histogram(velocities, bins=100)
bins = bins[:-1] + (bins[1] - bins[0])/2
hist = hist / np.sum(hist) # normalize
# plot the histogram
plt.bar(bins, hist, width=bins[1]-bins[0], alpha=0.5, color='gray')
p0 = [0.33, 0.1] + [0.33, 0.1, 20]
bounds = [[0,1],[0,np.inf]] + [[0,1],[0,np.inf],[0,max_velocity]]
bounds = list(zip(*bounds))
popt2, pcov = curve_fit(eg_model, bins, hist, maxfev=50000, p0=p0, bounds=bounds)
pred2 = eg_model(bins, *popt2)
plt.plot(bins, pred2, 'k-', linewidth=2)
# # plot the individual components
plt.plot(bins, exp(bins, *popt2[:2]), 'k--', label='Exponential')
plt.plot(bins, gauss(bins, *popt2[2:]), 'g--', label='Gaussian')
plt.xlabel('Velocity (mm/s)')
plt.ylabel('Probability')
plt.yscale('log')
plt.title('Velocity distribution (Exp + Gaussian)')
plt.ylim([1e-3, 1])
plt.legend(frameon=False)
plt.savefig('figures/velocity_distribution_exp_gaussian.png')
plt.show()


# compare the three models
plt.figure()
plt.plot(bins, hist, 'k-', linewidth=2, label='Data')
plt.plot(bins, pred1, 'r--', label='Exp + 2 Gaussians')
plt.plot(bins, pred2, 'g--', label='Exp + Gaussian')
plt.xlabel('Velocity (mm/s)')
plt.ylabel('Probability')
plt.yscale('log')
plt.ylim([1e-3, 1])
plt.title('Velocity distribution')
plt.legend()
plt.savefig('figures/velocity_distribution_comparison.png')
plt.show()

In [237]:
# create a function to automate the fitting of exponential + 2 gaussians model
def fit_data_egg(velocities, bins=None, max_velocity=10*3):
    # create a histogram
    if bins is None:
        bins = 100
    hist, bins = np.histogram(velocities, bins=bins)
    bins = bins[:-1] + (bins[1] - bins[0])/2
    hist = hist / np.sum(hist) # normalize
    # fit the data
    p0 = [0.33, 0.1] + [0.33, 0.1, 5] + [0.33, 0.1, 20]
    bounds = [[0,1],[0,np.inf]] + [[0,1],[0,np.inf],[0,max_velocity]]*2
    bounds = list(zip(*bounds))
    popt, _ = curve_fit(egg_model, bins, hist, maxfev=50000, p0=p0, bounds=bounds)
    pred = egg_model(bins, *popt)
    # resolve degeneracy by swapping the gaussians if the first gaussian has a higher mean
    if popt[4] > popt[7]:
        popt = popt[:2] + popt[5:] + popt[2:5]
    weights = [popt[0], popt[2], popt[5]]
    e_params = popt[1:2]
    g1_params = popt[3:5]
    g2_params = popt[6:]
    # return the parameters
    return weights, e_params, g1_params, g2_params, bins, hist, pred

# create a function to automate the fitting of exponential + 2 gaussians model
def fit_data_eg(velocities, bins=None, max_velocity=10*3):
    # create a histogram
    if bins is None:
        bins = 100
    hist, bins = np.histogram(velocities, bins=bins)
    bins = bins[:-1] + (bins[1] - bins[0])/2
    hist = hist / np.sum(hist) # normalize
    # fit the data
    p0 = [0.33, 0.1] + [0.33, 0.1, 20]
    bounds = [[0,1],[0,np.inf]] + [[0,1],[0,np.inf],[0,max_velocity]]
    bounds = list(zip(*bounds))
    popt, _ = curve_fit(eg_model, bins, hist, maxfev=50000, p0=p0, bounds=bounds)
    pred = eg_model(bins, *popt)
    weights = [popt[0], popt[2]]
    e_params = popt[1:2]
    g_params = popt[3:5]
    # return the parameters
    return weights, e_params, g_params, bins, hist, pred

In [238]:
# create a classifying function that returns the probability of the velocity belonging to the exponential distribution or the gaussian distribution
def classify_velocity(velocity, popt):
    a, b, c, d, e = popt
    p_exp = exp(velocity, a, b)
    p_gauss = gauss(velocity, c, d, e)
    p = p_exp + p_gauss
    return np.argmax([p_exp/p, p_gauss/p])
# classify the velocities
velocity_classification = [classify_velocity(v, popt2) for v in velocities]

In [239]:
distances = np.concatenate([track[track['velocity'] < max_velocity]['radial_distance'].values for track in tracklets])
# plot distance vs velocity
plt.scatter(distances, velocities, s=1, alpha=0.1, c = velocity_classification, cmap='coolwarm')
plt.xlabel('Distance from the center (mm)')
plt.ylabel('Velocity (mm/s)')
plt.show()

In [240]:
# show distribution of velocities for different binned distances
min_distance = np.min(distances)
max_distance = np.max(distances)
n_bins = 10
bins = np.linspace(min_distance, max_distance, n_bins)
bin_centers = (bins[:-1] + bins[1:])/2

# velocity bins
vbins = np.linspace(0, np.max(velocities), 100)
vbin_centers = (vbins[:-1] + vbins[1:])/2

# get the velocities for each bin and fit data with model 1 (gamma + 2 gaussians)
velocities_binned = []
histograms = []
predictions = []
weights = []
e_params = []
g_params = []
for i in range(n_bins-1):

    binned_velocities = velocities[(distances >= bins[i]) & (distances < bins[i+1])]
    w, e, g, _, hist, pred = fit_data_eg(binned_velocities, bins=vbins)

    # append to the list
    velocities_binned.append(binned_velocities)
    weights.append(w)
    e_params.append(e)
    g_params.append(g)
    histograms.append(hist)
    predictions.append(pred)

# plot the histograms
from matplotlib.transforms import Bbox
fig, _ = plt.subplots(figsize=(7, 6))
# add space to the right
fig.subplots_adjust(right=0.6, top=0.5)
axes = [fig.gca()]
for i in range(n_bins-2):
    axes.append(fig.add_subplot(111, sharex=axes[0], sharey=axes[0]))
# set the limits
axes[-1].set(ylim=[1e-3, 1], yscale='log', xlim=[0, max_velocity], xlabel='Velocity (mm/s)', ylabel='Probability')
# add the histograms
for i in range(n_bins-2, -1, -1):
    axes[i].bar(vbin_centers, histograms[i], width=vbin_centers[1]-vbin_centers[0], alpha=0.8, color=plt.cm.tab10(i), label=f'{bins[i]:.1f} - {bins[i+1]:.1f} mm')
    axes[i].plot(vbin_centers, predictions[i], 'k-', linewidth=2)
fig.legend(loc='lower right', bbox_to_anchor=(1, 0.01), frameon=False, title='Distance from the center', fontsize='small')
x_shift, y_shift = 0.04, 0.06
for i, ax in enumerate(axes[::-1]):
    ax.patch.set_visible(False)
    pos = ax.get_position()
    newpos = Bbox.from_bounds(pos.x0+i*x_shift, pos.y0+i*y_shift, pos.width, pos.height)
    ax.set_position(newpos)
    for sp in ["top", "right"]:
        ax.spines[sp].set_visible(False)
    if ax != axes[-1]:
        ax.tick_params(labelleft=False, left=False, labelbottom=False)
plt.savefig('figures/velocity_distribution_binned_distance_exp_gaussian.png')
plt.show()

# plot the weights
weights = np.array(weights)
weights = weights / np.sum(weights, axis=1)[:,None]
plt.figure()
plt.plot(bin_centers, weights[:,0], 'k-', label='Exponential')
plt.plot(bin_centers, weights[:,1], 'g-', label='Gaussian')
plt.xlabel('Distance from the center (mm)')
plt.ylabel('Weight')
plt.legend()
plt.title('Weights of the components')
plt.savefig('figures/weights_of_components.png')
plt.show()

# plot the exponential parameters
e_params = np.array(e_params)
plt.figure()
plt.plot(bin_centers, e_params[:,0], 'g-')
plt.xlabel('Distance from the center (mm)')
plt.ylabel('Scale')
plt.title('Exponential(Distance)')
plt.savefig('figures/exponential_parameter.png')
plt.show()

# plot the gaussian parameters
g_params = np.array(g_params)
# limit the standard deviation to a maximum value
g_params[:,0] = np.maximum(g_params[:,0], 2/max_velocity**2)
plt.figure()
plt.plot(bin_centers, g_params[:,1], '-', label='Mean Velocity (Gaussian)', color='lightblue')
plt.plot(bin_centers, np.sqrt(2/g_params[:,0]), '-', label='Standard deviation (Gaussian)', color='navy')
plt.xlabel('Distance from the center (mm)')
plt.ylabel('Parameter')
plt.legend()
plt.title('Gaussian Parameters')
plt.savefig('figures/gaussian_parameters.png')
plt.show()

In [241]:
# find the local minima in the mixed distribution to find the transition points
from scipy.signal import find_peaks

minima = []
for i in range(n_bins-1):
    pred = predictions[i]
    peaks, _ = find_peaks(-pred)
    if len(peaks) == 0:
        peaks = [np.argmax(pred)]
    minima.append(vbin_centers[peaks[0]])
minima = np.array(minima)


def state_classifier(velocity, distance):
    # classify the state of the fly based on the velocity and distance
    # get the distance bin
    bin_idx = np.digitize(distance, bins) - 1
    if bin_idx < 0:
        bin_idx = 0
    if bin_idx >= n_bins-1:
        bin_idx = n_bins-2

    # use the velocity to classify
    if velocity < minima[bin_idx]:
        return 0
    elif velocity < max_velocity:
        return 1
    else:
        return 2

# classify the velocities
state_classification = [state_classifier(v, d) for v, d in zip(velocities, distances)]

# plot the state classification
plt.figure()
plt.scatter(distances, velocities, c=state_classification, s=1, cmap='coolwarm', alpha=0.1)
plt.xlabel('Distance from the center (mm)')
plt.ylabel('Velocity (mm/s)')
plt.colorbar(label='State')
plt.title('State classification')
plt.savefig('figures/state_classification.png')
plt.show()


In [242]:
# apply classification to all tracklets
for track in tracklets:
    track['state'] = track['velocity']<2.5
    # track['state'] = [state_classifier(v, d) for v, d in zip(track['velocity'], track['radial_distance'])]
    # track['state'] = [np.int32(classify_velocity(v, popt2)) for v in track['velocity']]

In [244]:
tracklet = tracklets[0]

# assuming state 2 is the same as state 1, calculate the transition probabilities
# get the transition matrix at each distance bin
transition_matrix = np.zeros((len(bin_centers), 2, 2))
for track in tracklets:
    states = np.minimum(track['state'].values, 1)
    # digitize the radial distance
    distance_bins = np.digitize(track['radial_distance'], bins) - 1
    for i in range(1, len(states)):
        if distance_bins[i] < 0:
            distance_bins[i] = 0
        if distance_bins[i] >= n_bins-1:
            distance_bins[i] = n_bins-2
        transition_matrix[distance_bins[i], states[i-1], states[i]] += 1
# normalize the matrix to get the transition probabilities
for i in range(len(bin_centers)):
    transition_matrix[i] = transition_matrix[i] / np.sum(transition_matrix[i], axis=1)[:,None]
    # scale to rate according to the frame rate
    transition_matrix[i] = transition_matrix[i] * FPS

# plot the transition rates at each distance bin
plt.figure()
s_f = np.array([m[0,1] for m in transition_matrix])
f_s = np.array([m[1,0] for m in transition_matrix])
plt.plot(bin_centers, s_f, 'r-', label='Slow to Fast')
plt.plot(bin_centers, f_s, 'b-', label='Fast to Slow')
plt.xlabel('Distance from the center (mm)')
plt.ylabel('Transition rate (Hz)')
plt.legend()
plt.title('Transition rates')
plt.savefig('figures/transition_rates.png')
plt.show()



